In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import requests
import os

In [2]:
url = 'https://dadosabertos.aneel.gov.br/dataset/da3d8da5-bdc0-471a-b57e-09e49f557c9a/resource/0bf361e5-dc58-4380-95f0-b61a55ec3423/download/sigec-taxa-de-fiscalizacao.csv'
nome_arquivo = '../dataset/dados.csv'
os.makedirs("../dataset", exist_ok=True)

def baixa_dataset():
    if os.path.exists(nome_arquivo):
        print("arquivo já existe")
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))
    
    with open(nome_arquivo, 'wb') as file:
        downloaded = 0
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)
                downloaded += len(chunk)
                if total_size > 0:
                    percent = (downloaded * 100) / total_size
                    print(f"\rDownload: {percent:.1f}% ({downloaded:,}/{total_size:,} bytes)", end='')
if not os.path.exists(nome_arquivo):
    baixa_dataset()


In [3]:
nome_dataset = 'dados.csv'

df = pd.read_csv(f"../dataset/{nome_dataset}", delimiter=';', encoding='latin1')
df.head()

,DatGeracaoConjuntoDados,Codcvnarr,AnmArrecadacao,DatVencimentoTitulo,DatIncidenciaMultaMora,DatPagamentoTitulo,QtdDiasEmAtraso,NumCPFCNPJ,SigNomAgente,NomEmpreendimento,NumProcessoSIC,NumAutoInfracao,DscSituacaoArrecadacao,DscSituacaoCredito,VlrPcpPrvArr,VlrTotPvrArr,VlrTotPagArr,VlrTotDifPvrPagArr,VlrSelic
0,2025-11-08,858158,200511.0,2005-12-15,2005-12-16,2005-12-15,0.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Taíba,-,-,Quitada pela conciliação automática,Quitada,"3120,38","3120,38","3120,38",",00","2,01874493468739993"
1,2025-11-08,2940945,202010.0,2020-11-16,2020-11-17,2020-11-11,-5.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Eólica de Prainha,-,-,Quitada pela conciliação automática,Quitada,"2549,10","2549,10","2549,10",",00",",51130000000000000"
2,2025-11-08,858158,200701.0,2007-02-15,2007-02-16,2007-02-15,0.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Eólica de Prainha,-,-,Quitada pela conciliação automática,Quitada,"1205,08","1205,08","1205,08",",00","1,80029697158140180"
3,2025-11-08,2667718,201407.0,2014-08-25,2014-08-26,2014-08-20,-5.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Eólica de Prainha,-,-,Quitada pela conciliação automática,Quitada,"1568,77","1568,77","1568,77",",00","1,04650000000000000"
4,2025-11-08,2667718,201412.0,2015-01-15,2015-01-16,2015-01-14,-1.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Eólica de Prainha,-,-,Quitada pela conciliação automática,Quitada,"1568,77","1568,77","1568,77",",00","1,00050000000000000"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415207 entries, 0 to 415206
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   DatGeracaoConjuntoDados  415207 non-null  object 
 1   Codcvnarr                415207 non-null  int64  
 2   AnmArrecadacao           415206 non-null  float64
 3   DatVencimentoTitulo      415207 non-null  object 
 4   DatIncidenciaMultaMora   415207 non-null  object 
 5   DatPagamentoTitulo       405411 non-null  object 
 6   QtdDiasEmAtraso          405411 non-null  float64
 7   NumCPFCNPJ               415207 non-null  int64  
 8   SigNomAgente             415207 non-null  object 
 9   NomEmpreendimento        360948 non-null  object 
 10  NumProcessoSIC           415207 non-null  object 
 11  NumAutoInfracao          415207 non-null  object 
 12  DscSituacaoArrecadacao   415207 non-null  object 
 13  DscSituacaoCredito       407820 non-null  object 
 14  VlrP

In [5]:
colunas_numericas = [
    'VlrPcpPrvArr',
    'VlrTotPvrArr',
    'VlrTotPagArr',
    'VlrTotDifPvrPagArr',
    'VlrSelic'
]
def to_float_br(series):
    return (
        series.astype(str)
              .str.strip()
              .str.replace('.', '', regex=False)      # remove milhar
              .str.replace(',', '.', regex=False)     # converte decimal
              .replace(['', 'nan', 'None'], np.nan)   # valores vazios
              .astype(float)
    )
for col in colunas_numericas:
    df[col] = to_float_br(df[col])
df[colunas_numericas].dtypes

VlrPcpPrvArr          float64
VlrTotPvrArr          float64
VlrTotPagArr          float64
VlrTotDifPvrPagArr    float64
VlrSelic              float64
dtype: object

In [6]:
df.isnull().sum()

DatGeracaoConjuntoDados        0
Codcvnarr                      0
AnmArrecadacao                 1
DatVencimentoTitulo            0
DatIncidenciaMultaMora         0
DatPagamentoTitulo          9796
QtdDiasEmAtraso             9796
NumCPFCNPJ                     0
SigNomAgente                   0
NomEmpreendimento          54259
NumProcessoSIC                 0
NumAutoInfracao                0
DscSituacaoArrecadacao         0
DscSituacaoCredito          7387
VlrPcpPrvArr                   0
VlrTotPvrArr                   3
VlrTotPagArr                9795
VlrTotDifPvrPagArr             0
VlrSelic                       0
dtype: int64

In [7]:
df.drop(columns='NomEmpreendimento', inplace=True)
df.isnull().sum()

DatGeracaoConjuntoDados       0
Codcvnarr                     0
AnmArrecadacao                1
DatVencimentoTitulo           0
DatIncidenciaMultaMora        0
DatPagamentoTitulo         9796
QtdDiasEmAtraso            9796
NumCPFCNPJ                    0
SigNomAgente                  0
NumProcessoSIC                0
NumAutoInfracao               0
DscSituacaoArrecadacao        0
DscSituacaoCredito         7387
VlrPcpPrvArr                  0
VlrTotPvrArr                  3
VlrTotPagArr               9795
VlrTotDifPvrPagArr            0
VlrSelic                      0
dtype: int64

In [8]:
df['DatVencimentoTitulo'] = pd.to_datetime(df['DatVencimentoTitulo'], errors='coerce')
df['DatPagamentoTitulo'] = pd.to_datetime(df['DatPagamentoTitulo'], errors='coerce')

df['DiffDias'] = (df['DatPagamentoTitulo'] - df['DatVencimentoTitulo']).dt.days

df.loc[df['DatVencimentoTitulo'].isna(), 'QtdDiasEmAtraso'] = 0

df.loc[df['DatPagamentoTitulo'].isna(), 'QtdDiasEmAtraso'] = 100

df.loc[df['QtdDiasEmAtraso'] < 0, 'QtdDiasEmAtraso'] = 0

mask = (
    df['DiffDias'].notna() &
    (df['DiffDias'] != df['QtdDiasEmAtraso'])
)

df.loc[mask, 'QtdDiasEmAtraso'] = df['DiffDias'].clip(lower=0)
df.drop(columns='DiffDias', inplace=True)

In [9]:
df.isnull().sum()

DatGeracaoConjuntoDados       0
Codcvnarr                     0
AnmArrecadacao                1
DatVencimentoTitulo           0
DatIncidenciaMultaMora        0
DatPagamentoTitulo         9796
QtdDiasEmAtraso               0
NumCPFCNPJ                    0
SigNomAgente                  0
NumProcessoSIC                0
NumAutoInfracao               0
DscSituacaoArrecadacao        0
DscSituacaoCredito         7387
VlrPcpPrvArr                  0
VlrTotPvrArr                  3
VlrTotPagArr               9795
VlrTotDifPvrPagArr            0
VlrSelic                      0
dtype: int64

In [10]:
df['DscSituacaoCredito'].value_counts()


DscSituacaoCredito
Quitada                                  404936
Inscrição em Dívida Ativa                  1110
Encaminhamento à Procuradoria Federal       558
Execução Judicial                           539
Notificação                                 444
Parcelamento na PRF                         160
CADIN                                        73
Name: count, dtype: int64

In [11]:
cond = df['DscSituacaoCredito'].isna() & df['DatPagamentoTitulo'].isna() & df['VlrTotPagArr'].isna() 
qtde_linhas = cond.sum()
qtde_linhas
print(f"Quantidade de linhas que não tem pagamento e situação de crédito: {qtde_linhas}")

Quantidade de linhas que não tem pagamento e situação de crédito: 6811


In [12]:
cond = df['DscSituacaoCredito'].isna() & df['DatPagamentoTitulo'].isna() 
qtde_linhas = cond.sum()
qtde_linhas
print(f"Quantidade de linhas que não tem pagamento e situação de crédito: {qtde_linhas}")

Quantidade de linhas que não tem pagamento e situação de crédito: 6811


In [13]:
df['DscSituacaoCredito'].unique()

array(['Quitada', nan, 'Inscrição em Dívida Ativa', 'Notificação',
       'CADIN', 'Encaminhamento à Procuradoria Federal',
       'Execução Judicial', 'Parcelamento na PRF'], dtype=object)

In [14]:
#df['VlrTotPvrArr'] = df['VlrTotPvrArr'].str.replace(',', '.').astype(float)
#df['VlrTotDifPvrPagArr'] = df['VlrTotDifPvrPagArr'].str.replace(',', '.').astype(float)
df['VlrTotPagArr_Corregido'] = df['VlrTotPvrArr'] - df['VlrTotDifPvrPagArr']
mask = df['VlrTotPagArr'].isna()

df.loc[mask, 'VlrTotPagArr'] = df.loc[mask, 'VlrTotPagArr_Corregido']
df.drop(columns='VlrTotPagArr_Corregido', inplace=True)

In [15]:
mask = (
    df['DscSituacaoCredito'].isna() &
    df['DatPagamentoTitulo'].isna()
)

df.loc[mask, 'DscSituacaoCredito'] = 'Inscrição em Dívida Ativa'

In [16]:
df.isnull().sum()

DatGeracaoConjuntoDados       0
Codcvnarr                     0
AnmArrecadacao                1
DatVencimentoTitulo           0
DatIncidenciaMultaMora        0
DatPagamentoTitulo         9796
QtdDiasEmAtraso               0
NumCPFCNPJ                    0
SigNomAgente                  0
NumProcessoSIC                0
NumAutoInfracao               0
DscSituacaoArrecadacao        0
DscSituacaoCredito          576
VlrPcpPrvArr                  0
VlrTotPvrArr                  3
VlrTotPagArr                  3
VlrTotDifPvrPagArr            0
VlrSelic                      0
dtype: int64

In [17]:
cond = df['DatPagamentoTitulo'].notna() & df['VlrTotPagArr'].isna()
qtde_linhas = cond.sum()
qtde_linhas

np.int64(0)

In [18]:
df['fatura_paga'] = df['DatPagamentoTitulo'].notna().astype(int)
df['fatura_atrasado'] = (df['QtdDiasEmAtraso'] > 0).astype(int)

In [19]:
df.isnull().sum() 

DatGeracaoConjuntoDados       0
Codcvnarr                     0
AnmArrecadacao                1
DatVencimentoTitulo           0
DatIncidenciaMultaMora        0
DatPagamentoTitulo         9796
QtdDiasEmAtraso               0
NumCPFCNPJ                    0
SigNomAgente                  0
NumProcessoSIC                0
NumAutoInfracao               0
DscSituacaoArrecadacao        0
DscSituacaoCredito          576
VlrPcpPrvArr                  0
VlrTotPvrArr                  3
VlrTotPagArr                  3
VlrTotDifPvrPagArr            0
VlrSelic                      0
fatura_paga                   0
fatura_atrasado               0
dtype: int64

In [20]:
cond = df['DscSituacaoCredito'].isna() & df['QtdDiasEmAtraso'] > 0
qtde_linhas = cond.sum()
qtde_linhas

np.int64(284)

In [21]:
mask = (
    df['DscSituacaoCredito'].isna() &
    df['QtdDiasEmAtraso'] > 0
)

df.loc[mask, 'DscSituacaoCredito'] = 'Inscrição em Dívida Ativa'

In [22]:
df.isnull().sum()

DatGeracaoConjuntoDados       0
Codcvnarr                     0
AnmArrecadacao                1
DatVencimentoTitulo           0
DatIncidenciaMultaMora        0
DatPagamentoTitulo         9796
QtdDiasEmAtraso               0
NumCPFCNPJ                    0
SigNomAgente                  0
NumProcessoSIC                0
NumAutoInfracao               0
DscSituacaoArrecadacao        0
DscSituacaoCredito          292
VlrPcpPrvArr                  0
VlrTotPvrArr                  3
VlrTotPagArr                  3
VlrTotDifPvrPagArr            0
VlrSelic                      0
fatura_paga                   0
fatura_atrasado               0
dtype: int64

In [23]:
cond = (df['DscSituacaoCredito'].isna()) & (df['QtdDiasEmAtraso'] == 0) & (df['DatPagamentoTitulo'].notna())  
qtde_linhas = cond.sum()
qtde_linhas

np.int64(292)

In [24]:
cond = (df['DscSituacaoCredito'].isna()) & (df['QtdDiasEmAtraso'] == 0) & (df['DatPagamentoTitulo'].notna())  
df.loc[cond, 'DscSituacaoCredito'] = 'Quitada'

In [25]:
df.isnull().sum()

DatGeracaoConjuntoDados       0
Codcvnarr                     0
AnmArrecadacao                1
DatVencimentoTitulo           0
DatIncidenciaMultaMora        0
DatPagamentoTitulo         9796
QtdDiasEmAtraso               0
NumCPFCNPJ                    0
SigNomAgente                  0
NumProcessoSIC                0
NumAutoInfracao               0
DscSituacaoArrecadacao        0
DscSituacaoCredito            0
VlrPcpPrvArr                  0
VlrTotPvrArr                  3
VlrTotPagArr                  3
VlrTotDifPvrPagArr            0
VlrSelic                      0
fatura_paga                   0
fatura_atrasado               0
dtype: int64

In [26]:
df.drop(columns=['DatPagamentoTitulo'], inplace=True)

In [27]:
df_nulos = df[df.isna().any(axis=1)]
df_nulos


,DatGeracaoConjuntoDados,Codcvnarr,AnmArrecadacao,DatVencimentoTitulo,DatIncidenciaMultaMora,QtdDiasEmAtraso,NumCPFCNPJ,SigNomAgente,NumProcessoSIC,NumAutoInfracao,DscSituacaoArrecadacao,DscSituacaoCredito,VlrPcpPrvArr,VlrTotPvrArr,VlrTotPagArr,VlrTotDifPvrPagArr,VlrSelic,fatura_paga,fatura_atrasado
152339,2025-11-08,858158,201112.0,2012-01-16,2012-01-17,100.0,5345447000116,PIEDADE PIEDADE USINA GERADORA DE...,-,-,Cancelada,Inscrição em Dívida Ativa,4562.68,NaN,NaN,0.0,1.2659,0,1
215548,2025-11-08,858158,201407.0,2014-08-25,2014-08-26,100.0,5751762000143,CEISA - CENTRAL ENERGETICA ITAUNAS S/A,-,-,Cancelada,Inscrição em Dívida Ativa,862.82,NaN,NaN,0.0,1.0465,0,1
215574,2025-11-08,858158,201406.0,2014-07-15,2014-07-16,100.0,5751762000143,CEISA - CENTRAL ENERGETICA ITAUNAS S/A,-,-,Cancelada,Inscrição em Dívida Ativa,862.82,NaN,NaN,0.0,1.0552,0,1
228383,2025-11-08,858158,NaN,2010-06-23,2010-06-24,84.0,7386098000106,RAESA RIO AMAZONAS ENERGIA S/A,-,-,Quitada pela conciliação automática,Quitada,231.54,284.22,284.22,0.0,1.4367,1,1


Como a quantidade de registros com valores ausentes são apenas quatro linhas, decidimos então remover estes registros.

In [28]:
df.dropna(inplace=True)

In [29]:
df.isnull().sum()

DatGeracaoConjuntoDados    0
Codcvnarr                  0
AnmArrecadacao             0
DatVencimentoTitulo        0
DatIncidenciaMultaMora     0
QtdDiasEmAtraso            0
NumCPFCNPJ                 0
SigNomAgente               0
NumProcessoSIC             0
NumAutoInfracao            0
DscSituacaoArrecadacao     0
DscSituacaoCredito         0
VlrPcpPrvArr               0
VlrTotPvrArr               0
VlrTotPagArr               0
VlrTotDifPvrPagArr         0
VlrSelic                   0
fatura_paga                0
fatura_atrasado            0
dtype: int64

In [30]:
quantidade_ausente_NumProcessoSIC = df[df['NumProcessoSIC'] == ' - ']['NumProcessoSIC'].count()

In [31]:
quantidade_ausente_NumAutoInfracao = df[df['NumAutoInfracao'] == '-']['NumAutoInfracao'].count()

In [32]:
print(f"A porcentagem de valores ausentes na coluna 'NumProcessoSIC' é de {(quantidade_ausente_NumProcessoSIC / len(df)) * 100:.2f}%")
print(f"A porcentagem de valores ausentes na coluna 'NumAutoInfracao' é de {(quantidade_ausente_NumAutoInfracao / len(df)) * 100:.2f}%")

A porcentagem de valores ausentes na coluna 'NumProcessoSIC' é de 99.14%
A porcentagem de valores ausentes na coluna 'NumAutoInfracao' é de 100.00%


Por essas duas colunasnão apresentarem dados que apenas identificam o processo da infração, não são relevantes para o estudo do problema. Por isso, decidimos remover estas duas colunas.

In [33]:
df.drop(columns=['NumProcessoSIC', 'NumAutoInfracao'], inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 415203 entries, 0 to 415206
Data columns (total 17 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   DatGeracaoConjuntoDados  415203 non-null  object        
 1   Codcvnarr                415203 non-null  int64         
 2   AnmArrecadacao           415203 non-null  float64       
 3   DatVencimentoTitulo      415203 non-null  datetime64[ns]
 4   DatIncidenciaMultaMora   415203 non-null  object        
 5   QtdDiasEmAtraso          415203 non-null  float64       
 6   NumCPFCNPJ               415203 non-null  int64         
 7   SigNomAgente             415203 non-null  object        
 8   DscSituacaoArrecadacao   415203 non-null  object        
 9   DscSituacaoCredito       415203 non-null  object        
 10  VlrPcpPrvArr             415203 non-null  float64       
 11  VlrTotPvrArr             415203 non-null  float64       
 12  VlrTotPagArr         

In [35]:
df.head()

,DatGeracaoConjuntoDados,Codcvnarr,AnmArrecadacao,DatVencimentoTitulo,DatIncidenciaMultaMora,QtdDiasEmAtraso,NumCPFCNPJ,SigNomAgente,DscSituacaoArrecadacao,DscSituacaoCredito,VlrPcpPrvArr,VlrTotPvrArr,VlrTotPagArr,VlrTotDifPvrPagArr,VlrSelic,fatura_paga,fatura_atrasado
0,2025-11-08,858158,200511.0,2005-12-15,2005-12-16,0.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Quitada pela conciliação automática,Quitada,3120.38,3120.38,3120.38,0.0,2.018745,1,0
1,2025-11-08,2940945,202010.0,2020-11-16,2020-11-17,0.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Quitada pela conciliação automática,Quitada,2549.10,2549.10,2549.10,0.0,0.511300,1,0
2,2025-11-08,858158,200701.0,2007-02-15,2007-02-16,0.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Quitada pela conciliação automática,Quitada,1205.08,1205.08,1205.08,0.0,1.800297,1,0
3,2025-11-08,2667718,201407.0,2014-08-25,2014-08-26,0.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Quitada pela conciliação automática,Quitada,1568.77,1568.77,1568.77,0.0,1.046500,1,0
4,2025-11-08,2667718,201412.0,2015-01-15,2015-01-16,0.0,1027335000166,WOBBEN WINDPOWER INDUSTRIA E COMERCIO LTDA,Quitada pela conciliação automática,Quitada,1568.77,1568.77,1568.77,0.0,1.000500,1,0


## Conectando ao DuckDb

In [41]:
import duckdb

df_sql = df.copy()

duckdb.register("df_table", df_sql)


### CONSULTA SQL 1 — Conteúdo Nulo por Coluna

In [47]:
duckdb.sql("""
    SELECT
        COUNT(*) AS total_linhas,
        SUM(CASE WHEN Codcvnarr IS NULL THEN 1 ELSE 0 END) AS Codcvnarr_missing,
        SUM(CASE WHEN DatVencimentoTitulo IS NULL THEN 1 ELSE 0 END) AS DatVencimentoTitulo_missing,
        SUM(CASE WHEN VlrPcpPrvArr IS NULL THEN 1 ELSE 0 END) AS VlrPcpPrvArr_missing,
        SUM(CASE WHEN DscSituacaoCredito IS NULL THEN 1 ELSE 0 END) AS DscSituacaoCredito_missing
    FROM df_table;
""")


┌──────────────┬───────────────────┬─────────────────────────────┬──────────────────────┬────────────────────────────┐
│ total_linhas │ Codcvnarr_missing │ DatVencimentoTitulo_missing │ VlrPcpPrvArr_missing │ DscSituacaoCredito_missing │
│    int64     │      int128       │           int128            │        int128        │           int128           │
├──────────────┼───────────────────┼─────────────────────────────┼──────────────────────┼────────────────────────────┤
│       415203 │                 0 │                           0 │                    0 │                          0 │
└──────────────┴───────────────────┴─────────────────────────────┴──────────────────────┴────────────────────────────┘

“Identificamos a quantidade de valores ausentes por coluna. As colunas ‘NumProcessoSIC’ e ‘NumAutoInfracao’ mostraram proporções muito altas de valores faltantes, justificando sua remoção no pipeline de limpeza.”

### CONSULTA SQL 2 — Distribuição das Situações de Crédito

In [43]:
duckdb.sql("""
    SELECT 
        DscSituacaoCredito,
        COUNT(*) AS quantidade,
        ROUND(100.0 * COUNT(*) / (SELECT COUNT(*) FROM df_table), 2) AS porcentagem
    FROM df_table
    GROUP BY DscSituacaoCredito
    ORDER BY quantidade DESC;
""")


┌───────────────────────────────────────┬────────────┬─────────────┐
│          DscSituacaoCredito           │ quantidade │ porcentagem │
│                varchar                │   int64    │   double    │
├───────────────────────────────────────┼────────────┼─────────────┤
│ Quitada                               │     405227 │        97.6 │
│ Inscrição em Dívida Ativa             │       8202 │        1.98 │
│ Encaminhamento à Procuradoria Federal │        558 │        0.13 │
│ Execução Judicial                     │        539 │        0.13 │
│ Notificação                           │        444 │        0.11 │
│ Parcelamento na PRF                   │        160 │        0.04 │
│ CADIN                                 │         73 │        0.02 │
└───────────────────────────────────────┴────────────┴─────────────┘

“A maioria das linhas pertence à categoria X. Essa distribuição ajudou a identificar valores inconsistentes e orientar correções nas regras de imputação da coluna DscSituacaoCredito.”

### CONSULTA SQL 3 — Outliers em Valores Monetários

In [44]:
duckdb.sql("""
    SELECT 
        AVG(VlrPcpPrvArr) AS media,
        STDDEV(VlrPcpPrvArr) AS desvio,
        MIN(VlrPcpPrvArr) AS minimo,
        MAX(VlrPcpPrvArr) AS maximo
    FROM df_table;
""")


┌────────────────────┬────────────────────┬────────┬────────────┐
│       media        │       desvio       │ minimo │   maximo   │
│       double       │       double       │ double │   double   │
├────────────────────┼────────────────────┼────────┼────────────┤
│ 32263.696954742176 │ 138765.68327916114 │    0.0 │ 4079708.62 │
└────────────────────┴────────────────────┴────────┴────────────┘

“Detectamos disparidades grandes entre mínimo e máximo em VlrPcpPrvArr, indicando potenciais outliers. Isso orientou a criação de gráficos boxplot para verificação visual.”

### CONSULTA SQL 4 — Verificação de Datas Inconsistentes

In [45]:
duckdb.sql("""
    SELECT 
        COUNT(*) AS datas_negativas
    FROM df_table
    WHERE DatVencimentoTitulo < '1900-01-01';
""")

┌─────────────────┐
│ datas_negativas │
│      int64      │
├─────────────────┤
│               0 │
└─────────────────┘

“Nenhuma data anômala abaixo de 1900 foi detectada após a limpeza, confirmando consistência do campo temporal.”

### CONSULTA SQL 5 — Médias por Situação de Crédito

In [46]:
duckdb.sql("""
    SELECT 
        DscSituacaoCredito,
        AVG(VlrPcpPrvArr) AS media_valor,
        AVG(VlrTotPagArr) AS media_pago
    FROM df_table
    GROUP BY DscSituacaoCredito
    ORDER BY media_valor DESC;
""")


┌───────────────────────────────────────┬────────────────────┬───────────────────┐
│          DscSituacaoCredito           │    media_valor     │    media_pago     │
│                varchar                │       double       │      double       │
├───────────────────────────────────────┼────────────────────┼───────────────────┤
│ Quitada                               │  32651.96026604803 │ 32720.25413316452 │
│ Inscrição em Dívida Ativa             │  19436.37488539384 │ 46.03999146549622 │
│ Parcelamento na PRF                   │  6804.983562500003 │               0.0 │
│ Notificação                           │ 3857.6213063063105 │               0.0 │
│ Encaminhamento à Procuradoria Federal │  2321.052007168458 │               0.0 │
│ Execução Judicial                     │ 1776.3563821892417 │               0.0 │
│ CADIN                                 │   774.422191780822 │               0.0 │
└───────────────────────────────────────┴────────────────────┴───────────────────┘

“Diferenças claras surgem entre categorias de crédito, validando a relevância analítica da coluna na modelagem futura.”